In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
import pandas as pd

df = pd.read_csv("spam.csv", encoding=encoding)
df.head(5)

,Category,Message,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

              Unnamed: 2         \
         freq      count unique   
Category                          
ham        30         45     39   
spam        4          5      4   

                                                                 Unnamed: 3  \
                                                        top freq      count   
Category                                                                      
ham        bt not his girlfrnd... G o o d n i g h t . . .@"    3         10   
spam                                            PO Box 5249    2          2   

                                            Unnamed: 4                       
         unique                    top freq      count unique      top freq  
Category                                                                     
ham           9                     GE    2          6      5  GNT:-)"    2  
spam          1   MK17 92H. 450Ppw 16"    2          0      0      NaN  NaN

In [ ]:
df['Category'].value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [ ]:
747/4825

0.15481865284974095

In [ ]:
df_spam = df[df['Category']=='spam']
df_spam.shape

(747, 5)

In [ ]:
df_ham = df[df['Category']=='ham']
df_ham.shape

(4825, 5)

In [ ]:
df_ham_downsampled = df_ham.sample(df_spam.shape[0])
df_ham_downsampled.shape

(747, 5)

In [ ]:
df_balanced = pd.concat([df_spam, df_ham_downsampled])
df_balanced.shape

(1494, 5)

In [ ]:
df_balanced['Category'].value_counts()

spam    747
ham     747
Name: Category, dtype: int64

In [ ]:
df_balanced.sample(5)

,Category,Message,Unnamed: 2,Unnamed: 3,Unnamed: 4
671,spam,SMS. ac sun0819 posts HELLO:\You seem cool,"wanted to say hi. HI!!!\"" Stop? Send STOP to ...",NaN,NaN
3139,spam,sexy sexy cum and text me im wet and warm and ...,NaN,NaN,NaN
3979,spam,ringtoneking 84484,NaN,NaN,NaN
950,ham,"Awesome, lemme know whenever you're around",NaN,NaN,NaN
2057,ham,"Nothing, i got msg frm tht unknown no..",NaN,NaN,NaN


In [ ]:
df_balanced['spam'] = df_balanced['Category'].apply(lambda x: 1 if x == 'spam' else 0)
df_balanced.sample(10)

,Category,Message,Unnamed: 2,Unnamed: 3,Unnamed: 4,spam
1622,spam,U have a secret admirer who is looking 2 make ...,NaN,NaN,NaN,1
4235,spam,U can WIN å£100 of Music Gift Vouchers every w...,NaN,NaN,NaN,1
5456,spam,\For the most sparkling shopping breaks from 4...,NaN,NaN,NaN,1
5294,spam,XMAS iscoming & ur awarded either å£500 CD gif...,NaN,NaN,NaN,1
3421,spam,Freemsg: 1-month unlimited free calls! Activat...,NaN,NaN,NaN,1
2969,ham,U should make a fb list,NaN,NaN,NaN,0
629,spam,Todays Vodafone numbers ending with 4882 are s...,NaN,NaN,NaN,1
1639,spam,FreeMsg:Feelin kinda lnly hope u like 2 keep m...,NaN,NaN,NaN,1
3314,spam,FREE MESSAGE Activate your 500 FREE Text Messa...,NaN,NaN,NaN,1
727,ham,"Alright omw, gotta change my order to a half8th",NaN,NaN,NaN,0


In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df_balanced['Message'], df_balanced['spam'], stratify=df_balanced['spam'])

In [ ]:
x_train.head(4)

3740                                        2/2 146tf150p
3882    Gumby's has a special where a  &lt;#&gt; \ che...
1507    Wen ur lovable bcums angry wid u, dnt take it ...
4405    As one of our registered subscribers u can ent...
Name: Message, dtype: object

In [ ]:
bert_preprocess = hub.KerasLayer('https://kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-preprocess/versions/3')
bert_encoder = hub.KerasLayer("https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/bert-en-uncased-l-12-h-768-a-12/versions/2")

In [ ]:
def get_sentence_embedding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embedding([
    "500$ discount. hurry up",
    "Bhavin, are you up for a volleybal game tomorrow?"
])

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[ 0.07327941, -0.24222316,  0.77541345, ..., -0.77640074,
         0.37399358,  0.9943149 ],
       [-0.14637987, -0.47830647, -0.6007853 , ..., -0.9089969 ,
         0.45683342,  0.9887983 ]], dtype=float32)>

In [ ]:
e = get_sentence_embedding([
    "banana",
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates",
])

In [ ]:
e

<tf.Tensor: shape=(6, 768), dtype=float32, numpy=
array([[ 0.26732698, -0.19118263,  0.08356228, ..., -0.6183353 ,
         0.00336398,  0.999784  ],
       [ 0.36010486,  0.14107595,  0.91148806, ..., -0.89940435,
        -0.11671587,  0.99929357],
       [ 0.2476819 , -0.2332346 ,  0.5440166 , ..., -0.42085278,
         0.3458959 ,  0.99843556],
       [ 0.38079622,  0.28546444,  0.99195176, ..., -0.7380235 ,
         0.09374347,  0.99240774],
       [-0.11955263, -0.5513143 ,  0.9718947 , ..., -0.881357  ,
         0.6639831 ,  0.97707385],
       [-0.19652584, -0.14198987,  0.99629825, ..., -0.9222704 ,
         0.34902468,  0.96650016]], dtype=float32)>

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity([e[3]], [e[4]])

array([[0.629478]], dtype=float32)

In [ ]:
# Bert Layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

#Neural Network Layers

l = tf.keras.layers.Dropout(0.1, name='dropout')(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(l)

# construct final model
model = tf.keras.Model(inputs=[text_input], outputs=[l])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer_4 (KerasLayer)  {'input_mask': (None, 128)   0         ['text[0][0]']                
                             , 'input_word_ids': (None,                                           
                              128),                                                               
                              'input_type_ids': (None,                                            
                             128)}                                                                
                                                                                              

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics = METRICS)

In [ ]:
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
35/35 [==============================] - 499s 14s/step - loss: 0.5635 - accuracy: 0.7429 - precision: 0.7259 - recall: 0.7804
Epoch 2/10
35/35 [==============================] - 561s 16s/step - loss: 0.3689 - accuracy: 0.9080 - precision: 0.8827 - recall: 0.9411
Epoch 3/10
35/35 [==============================] - 491s 14s/step - loss: 0.2799 - accuracy: 0.9321 - precision: 0.9116 - recall: 0.9571
Epoch 4/10
35/35 [==============================] - 481s 14s/step - loss: 0.2319 - accuracy: 0.9536 - precision: 0.9425 - recall: 0.9661
Epoch 5/10
35/35 [==============================] - 480s 14s/step - loss: 0.2049 - accuracy: 0.9509 - precision: 0.9407 - recall: 0.9625
Epoch 6/10
35/35 [==============================] - 479s 14s/step - loss: 0.1831 - accuracy: 0.9571 - precision: 0.9491 - recall: 0.9661
Epoch 7/10
35/35 [==============================] - 480s 14s/step - loss: 0.1713 - accuracy: 0.9563 - precision: 0.9538 - recall: 0.9589
Epoch 8/10
35/35 [=======================

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
import numpy as np
y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion matrix(y_test, y_predicted)
cm

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sn
sn.heatmap(cm, annot=True, fmt = 'd')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
print(classification_report(y_test, y_predicted))